In [1]:
import os

In [2]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/shrey_98/Desktop/MLOps_CNNClassifier'

In [46]:
#update entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int 
    params_is_augmentation: bool
    params_image_size: list

    

In [47]:
from MLOPSCNNCLASSIFIER.constants import *
from MLOPSCNNCLASSIFIER.utils.common import read_yaml,create_directories
import tensorflow as tf

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "KIDNEY_CT_SCAN")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [59]:
import os
import urllib.request as request
from zipfile import ZipFile 
import tensorflow as tf
import time

In [64]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split=0.20
        )

        dataflow_kwargs= dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory=  self.config.training_data,
            subset  ="validation",
            shuffle= False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator= valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
    

    def train(self):
        self.steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps

        )
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        
    


In [65]:
try:
    config= ConfigurationManager()
    training_config= config.get_training_config()
    training=Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-03-20 17:00:40,237: INFO: common: YAML file: config/config.yaml loaded successfully]
[2024-03-20 17:00:40,239: INFO: common: YAML file: params.yaml loaded successfully]
[2024-03-20 17:00:40,240: INFO: common: Directory created at artifacts]
[2024-03-20 17:00:40,240: INFO: common: Directory created at artifacts/training]
Found 1471 images belonging to 2 classes.
Found 5889 images belonging to 2 classes.
Epoch 1/10
368/368 [==============================] - 1008s 3s/step - loss: 10.7637 - accuracy: 0.6172 - val_loss: 13.1835 - val_accuracy: 0.6971
Epoch 2/10
368/368 [==============================] - 1111s 3s/step - loss: 6.6003 - accuracy: 0.6923 - val_loss: 17.0148 - val_accuracy: 0.6971
Epoch 3/10
368/368 [==============================] - 1136s 3s/step - loss: 4.2807 - accuracy: 0.7422 - val_loss: 16.7317 - val_accuracy: 0.6971
Epoch 4/10
368/368 [==============================] - 2654s 7s/step - loss: 3.7069 - accuracy: 0.7609 - val_loss: 1.2661 - val_accuracy: 0.8249
Epoch 5/

/opt/anaconda3/envs/venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
